In [12]:
import time
import serial
import numpy as np
import sys
import os
import asyncio
from multiprocessing import Process
import threading
from scipy import constants
import itertools

import binascii
import datetime
import logging

Rack, Rackcommands, and Vial are originally from platform_setup.py

In [13]:
class Rack():
    """Representation for a Rack within the flow setup."""
    def __init__(self, array_dimensions, offset_x, offset_y, vial2vial_x, vial2vial_y, groundlevel_height):
        self.array_dimensions=array_dimensions
        self.offset_x=offset_x
        self.offset_y=offset_y
        self.vial2vial_x=vial2vial_x
        self.vial2vial_y=vial2vial_y
        self.groundlevel_height=groundlevel_height    
    
    def get_vial_indices(self, vial_position, array_order, tolerance):
        """get indices of a specific vial in a rack with a certain order of the vials
        :returns: a tuple of (i,j) with i=vial-position along x-axis, and j=vial-position along y-axis
        TODO: verify that input is valid type, array dimensions and validation of the inputted values
        """
        indices=np.where(array_order==vial_position)
        # print(str(f'indices are: {indices}'))
        if len(indices)==2 and len(indices[0])==1:
            logger.debug(f'a unique vial number was chosen: {vial_position}, with indices i={indices[0]}, j={indices[1]}')
            return indices
        elif len(indices)==2 and len(indices[0])==0 and tolerance.lower()=='no':                #tolerance settings
            #REMOVE THIS STATEMENT!!!
            sys.exit(f'fatal error: zero vials with position number {vial_position}')                        #REMOVE THIS STATEMENT!!!
        else:
            logger.warning(f'warning: multiple vials with position number {vial_position}')
            return indices

In [14]:
# Configure the logger - added to work in the Jupyter Notebook
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class Rackcommands(): 
    """Representation for Commands connected to the Rack of the flow setup."""

    def __init__(self,rack,rack_order,rack_position,rack_position_offset_x,rack_position_offset_y):
        self.rack=rack
        self.rack_order=rack_order
        self.rack_position=rack_position
        self.rack_position_offset_x=rack_position_offset_x
        self.rack_position_offset_y=rack_position_offset_y
        
    def get_xy_command(self, vial_pos: int, tolerance: str = 'no') -> str: #speed 125mm/s, force 100%
        """returns a str object command suitable for the liquid handler gx-241"""
        
        index_y, index_x = self.rack.get_vial_indices(vial_pos, self.rack_order, tolerance)
        
        if len(index_x)==len(index_y):
            command=[]
            for i in range(len(index_x)):
                i_x=index_x[i]
                i_y=index_y[i]
                distance_x=self.rack.offset_x + self.rack.vial2vial_x * i_x + (self.rack_position-1)*self.rack_position_offset_x    
                distance_y=self.rack.offset_y + self.rack.vial2vial_y * i_y + (self.rack_position-1)*self.rack_position_offset_y    
                command.append(str(f'X{distance_x}/{distance_y}'))
            return command
        else:
            logger.error("error: len(index_x) != len(index_y) ")

In [15]:
class Vial():
    """Representation for a Vial within the flow setup."""
    def __init__(self,vial_volume_max,vial_usedvolume_max,vial_height,vial_free_depth):
        self.vial_volume_max=vial_volume_max                #volume in mL
        self.vial_usedvolume_max=vial_usedvolume_max        #volume in mL
        self.vial_height=vial_height                        #height in mm
        self.vial_free_depth=vial_free_depth                #depth in mm
        self.sum_liquid_level = 0

This code is originally from run.py - the function get_automation_setup

In [16]:
rack_position_offset_x=92       #distance in mm between rack_position=1 and =2 on x-axis
rack_position_offset_y=0        #distance in mm between rack_position=1 and =2 on y-axis

############################# RACK 1 DEFINITION #################################

# From platform_setup.py 
rack1 = Rack([4,16], 7, 39.25, 18, 13.75, 82) # groundlevel_height assumed the minimum Z 

#  array_dimensions, offset_x, offset_y, vial2vial_x, vial2vial_y, groundlevel_height

# Previous vial2vialx = (2.11+15.6)
# Previous vial2vial7 = (2.72+15.6+0.35)

array_order1 = np.array([      #user is obliged to define a integer number i>=1 for each vial in the rack in ascending order 
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10,11,12],
    [13,14,15,16],
    [17,18,19,20],
    [21,22,23,24],     
    [25,26,27,28],     
    [29,30,31,32],
    [33,34,35,36],
    [37,38,39,40],
    [41,42,43,44],
    [45,46,47,48],
    [49,50,51,52],
    [53,54,55,56],
    [57,58,59,60],
    [61,62,63,64]        
    ])
    
rack_pos1=1

global rack1_commands

# Not sure what rack_position_offset_x/y are for x=92 and y=0

rack1_commands = Rackcommands(rack1, array_order1, rack_pos1, rack_position_offset_x, rack_position_offset_y)

global vial_selfmade

vial_selfmade = Vial(1.5, 1, 33, 31.08)

In [17]:
# It works!
rack1_commands.get_xy_command(36)

2024-10-16 14:48:57,599 - DEBUG - a unique vial number was chosen: 36, with indices i=[8], j=[3]


['X61/149.25']

## Method for collecting SPKA profiles

Take this code over to the gx-241 control v1 sheet to see if it works

In [16]:
# Define kinetic screen
no_sub_A = 2      # First Column
no_sub_B = 2      # Second Column
no_cat = 1        # Third Column

# Create lists for each
sub_A_list = [1 + 4 * i for i in range(no_sub_A)]
sub_B_list = [2 + 4 * i for i in range(no_sub_B)]
cat_list = [3 + 4 * i for i in range(no_cat)]

#print(sub_A_array, sub_B_array, cat_array)

In [17]:
# Generate all possible combinations
all_combinations = list(itertools.product(sub_A_list, sub_B_list, cat_list))

# Reorder combinations so that all combinations with the same elements
# in sub_B_array and cat_array are grouped together
spka_combinations = []

# Iterate through all unique combinations of sub_B_array and cat_array
# This will go through all sub_A first, then sub_B, then cat
for c in cat_list:
    for b in sub_B_list:
        for a in sub_A_list:
            spka_combinations.append((a, b, c))
            
spka_combinations

[(1, 2, 3), (5, 2, 3), (1, 6, 3), (5, 6, 3)]

In [20]:
# Iterate over each tuple in the list (each separate kinetic profile)
for tuple in spka_combinations:
    
    # Iterate over each number in the tuple (each reagent in an experiment)
    for element in tuple:
        
        # Find the xy position of the vial        
        vial_xy_pos = rack1_commands.get_xy_command(element)
        
        ##### TO DO #####
        
        # Display vial number - must be a two digit integer
        # g.bCommand(f"W{element:02}")
        
        # Go to Vial
        # g.bCommand(vial_xy_pos)

        # Needle Down - define this distance somewhere above!
        # g.bCommand(needle_down)

        # Run the pump
        # This will be a while away...
        # How to make the autosampler idle while this is happening?

        # Needle Up
        #g.bCommand(Z0)

        # Log the vial and the amount taken
        # Add a logger...

        # Create air gap
        # Create a tiny airgap with the pump, will this be necessary?

        print(vial_xy_pos)
        
    # Go to the DIM and inject
    # Blank the display
    #g.bCommand('WBB')
    
    # For now, let's just go to Home
    #g.bConnect('H')

2024-06-25 21:15:37,304 - DEBUG - a unique vial number was chosen: 1, with indices i=[0], j=[0]
2024-06-25 21:15:37,305 - DEBUG - a unique vial number was chosen: 2, with indices i=[0], j=[1]
2024-06-25 21:15:37,306 - DEBUG - a unique vial number was chosen: 3, with indices i=[0], j=[2]
2024-06-25 21:15:37,307 - DEBUG - a unique vial number was chosen: 5, with indices i=[1], j=[0]
2024-06-25 21:15:37,308 - DEBUG - a unique vial number was chosen: 2, with indices i=[0], j=[1]
2024-06-25 21:15:37,309 - DEBUG - a unique vial number was chosen: 3, with indices i=[0], j=[2]
2024-06-25 21:15:37,310 - DEBUG - a unique vial number was chosen: 1, with indices i=[0], j=[0]
2024-06-25 21:15:37,311 - DEBUG - a unique vial number was chosen: 6, with indices i=[1], j=[1]
2024-06-25 21:15:37,312 - DEBUG - a unique vial number was chosen: 3, with indices i=[0], j=[2]
2024-06-25 21:15:37,312 - DEBUG - a unique vial number was chosen: 5, with indices i=[1], j=[0]
2024-06-25 21:15:37,313 - DEBUG - a uniq

['X8.3/36.0']
['X26.3/36.0']
['X44.3/36.0']
['X8.3/49.75']
['X26.3/36.0']
['X44.3/36.0']
['X8.3/36.0']
['X26.3/49.75']
['X44.3/36.0']
['X8.3/49.75']
['X26.3/49.75']
['X44.3/36.0']
